## Process JUMP phenotypic profiles

We applied the AreaShape only class-balanced multiclass elastic net logistic regression model to all single-cell profiles in the JUMP dataset.

We then performed a series of KS tests to identify how different treatment distributions of all phenotype probabilities differed from controls.

See https://github.com/WayScience/JUMP-single-cell/tree/main/3.analyze_data#analyze-predicted-probabilities for complete details.

Here, we perform the following:

1. Load in this data from the JUMP-single-cell repo
2. Summarize replicate KS test metrics (mean value) and align across cell types and time variables
3. Explore the top results per phenotype/treatment_type/model_type
4. Convert it to wide format

This wide format represents a "phenotypic profile" which we can use similarly as an image-based morphology profile.

We also fit a UMAP to this phenotypic profile for downstream visualization.

In [1]:
import pathlib
from typing import List
import pandas as pd

import umap

In [2]:
def umap_phenotype(
    phenotype_df: pd.DataFrame,
    feature_columns: List[str],
    metadata_columns: List[str],
    n_components: int,
    random_seed: int,
    model_type: str
) -> pd.DataFrame:
    """
    Fit a UMAP (Uniform Manifold Approximation and Projection) model on the provided phenotype profile and return a transformed DataFrame with metadata.

    Parameters:
    - phenotype_df (pd.DataFrame): DataFrame containing the phenotype profile with both feature and metadata columns.
    - feature_columns (List[str]): List of column names in phenotype_df that represent the features to be used for UMAP embedding.
    - metadata_columns (List[str]): List of column names in phenotype_df that represent metadata to be retained in the output.
    - n_components (int): Number of dimensions for the UMAP embedding.
    - random_seed (int): Random seed for reproducibility of the UMAP model.
    - model_type (str): Identifier for the model type, to be added as a column in the output DataFrame.

    Returns:
    - umap_embeddings_with_metadata_df (pd.DataFrame): DataFrame with UMAP embeddings and specified metadata columns, including an additional 'model_type' column.
    """
    
    # Initialize UMAP
    umap_fit = umap.UMAP(random_state=random_seed, n_components=n_components)
    
    # Fit UMAP and convert to pandas DataFrame
    embeddings = pd.DataFrame(
        umap_fit.fit_transform(phenotype_df.loc[:, feature_columns]),
        columns=[f"UMAP{x}" for x in range(0, n_components)],
    )
    
    # Combine with metadata
    umap_embeddings_with_metadata_df = pd.concat([phenotype_df.loc[:, metadata_columns], embeddings], axis=1).assign(model_type=model_type)
    return umap_embeddings_with_metadata_df

In [3]:
# Set file paths
# JUMP phenotype probabilities from AreaShape model
commit = "4225e427fd9da59159de69f53be65c31b4d4644a"

url = "https://github.com/WayScience/JUMP-single-cell/raw"
file = "3.analyze_data/class_balanced_well_log_reg_comparison_results/class_balanced_well_log_reg_areashape_model_comparisons.parquet"

jump_sc_pred_file = f"{url}/{commit}/{file}"

# Set constants
n_top_results_to_explore = 100

In [4]:
# Set output files
output_dir = "jump_phenotype_profiles"

cell_type_time_comparison_file = pathlib.Path(output_dir, "jump_compare_cell_types_and_time_across_phenotypes.tsv.gz")
top_results_summary_file = pathlib.Path(output_dir, "jump_most_significant_phenotype_enrichment.tsv")
final_jump_phenotype_file = pathlib.Path(output_dir, "jump_phenotype_profiles.tsv.gz")
shuffled_jump_phenotype_file = pathlib.Path(output_dir, "jump_phenotype_profiles_shuffled.tsv.gz")

jump_umap_file = pathlib.Path(output_dir, "jump_phenotype_profiling_umap.tsv.gz")

## Load and process data

In [5]:
# Load KS test results and drop uninformative columns
jump_pred_df = (
    pd.read_parquet(jump_sc_pred_file)
    .drop(columns=["statistical_test", "comparison_metric"])
)

print(jump_pred_df.shape)
jump_pred_df.head()

(485370, 11)


,comparison_metric_value,p_value,Metadata_Plate,treatment,Metadata_model_type,treatment_type,Metadata_Well,Cell_type,Time,cell_count,phenotype
0,0.091654,0.013130,BR00117002,ABL1,final,crispr,C01,A549,144,592,ADCCM
1,0.118823,0.000441,BR00117002,ABL1,final,crispr,C01,A549,144,592,Anaphase
2,0.121319,0.000273,BR00117002,ABL1,final,crispr,C01,A549,144,592,Apoptosis
3,0.054403,0.332314,BR00117002,ABL1,final,crispr,C01,A549,144,592,Binuclear
4,0.030717,0.931704,BR00117002,ABL1,final,crispr,C01,A549,144,592,Elongated


In [6]:
# Process data to match treatments and scores across cell types
jump_pred_compare_df = (
    jump_pred_df
    # Summarize replicate scores
    .groupby([
        "Cell_type",
        "Time",
        "treatment",
        "treatment_type",
        "Metadata_model_type",
        "phenotype"
    ])
    .agg({
        "comparison_metric_value": "mean",
        "p_value": "mean"
    })
    .reset_index()
    # Compare per treatment scores across cell types
    .pivot(
        index=[
            "treatment",
            "treatment_type",
            "Time",
            "phenotype",
            "Metadata_model_type"
        ],
        columns="Cell_type",
        values=[
            "comparison_metric_value",
            "p_value"
        ]
    )
    .reset_index()
)

# Clen up column names
jump_pred_compare_df.columns = jump_pred_compare_df.columns.map(lambda x: '_'.join(filter(None, x)))

# Output file
jump_pred_compare_df.to_csv(cell_type_time_comparison_file, sep="\t", index=False)

print(jump_pred_compare_df.shape)
jump_pred_compare_df.head()

(37320, 9)


,treatment,treatment_type,Time,phenotype,Metadata_model_type,comparison_metric_value_A549,comparison_metric_value_U2OS,p_value_A549,p_value_U2OS
0,1-EBIO,compound,24,ADCCM,final,0.042401,0.049459,0.142509,0.530250
1,1-EBIO,compound,24,ADCCM,shuffled,0.027813,0.040563,0.595240,0.539444
2,1-EBIO,compound,24,Anaphase,final,0.036235,0.064360,0.343061,0.285088
3,1-EBIO,compound,24,Anaphase,shuffled,0.033513,0.048337,0.461138,0.346912
4,1-EBIO,compound,24,Apoptosis,final,0.033457,0.072364,0.468468,0.100510


In [7]:
# Focus on the top results for downstream interpretation
jump_focused_top_results_df = (
    jump_pred_df
    .query("Metadata_model_type == 'final'")
    .groupby(["Metadata_model_type", "treatment_type", "Cell_type", "Time", "phenotype"])
    .apply(lambda x: x.nsmallest(n_top_results_to_explore, "p_value"))
    .reset_index(drop=True)
)

jump_focused_top_results_df.to_csv(top_results_summary_file, sep="\t", index=False)

print(jump_focused_top_results_df.shape)
jump_focused_top_results_df.head()

(18000, 11)


,comparison_metric_value,p_value,Metadata_Plate,treatment,Metadata_model_type,treatment_type,Metadata_Well,Cell_type,Time,cell_count,phenotype
0,0.509470,2.854138e-58,BR00116992,CYT-997,final,compound,C09,A549,24,489,ADCCM
1,0.508134,3.137002e-54,BR00116993,CYT-997,final,compound,E06,A549,24,456,ADCCM
2,0.398279,1.072331e-53,BR00116993,fludarabine-phosphate,final,compound,N12,A549,24,753,ADCCM
3,0.468131,4.861256e-51,BR00116993,CYT-997,final,compound,C09,A549,24,509,ADCCM
4,0.495328,7.660156e-51,BR00116992,CYT-997,final,compound,E06,A549,24,450,ADCCM


## Summarize data

In [8]:
# How many unique plates?
jump_pred_df.Metadata_Plate.nunique()

51

In [9]:
# How many different individual treatments?
jump_pred_df.query("Metadata_model_type == 'final'").treatment_type.value_counts()

compound    113505
crispr       86130
orf          43050
Name: treatment_type, dtype: int64

In [10]:
# How many unique treatments per treatment type?
jump_pred_df.groupby("treatment_type").treatment.nunique()

treatment_type
compound    302
crispr      160
orf         160
Name: treatment, dtype: int64

In [11]:
# How many treatments with phenotype predictions?
jump_pred_df.query("Metadata_model_type == 'final'").phenotype.value_counts()

ADCCM                 16179
Anaphase              16179
Apoptosis             16179
Binuclear             16179
Elongated             16179
Grape                 16179
Hole                  16179
Interphase            16179
Large                 16179
Metaphase             16179
MetaphaseAlignment    16179
OutOfFocus            16179
Polylobed             16179
Prometaphase          16179
SmallIrregular        16179
Name: phenotype, dtype: int64

## Convert data to phenotypic profiles

In [12]:
metadata_columns = [
    "Metadata_Plate",
    "treatment",
    "treatment_type",
    "Cell_type",
    "Time",
    "Metadata_Well",
    "cell_count"
]

In [13]:
jump_wide_final_df = (
    jump_pred_df
    .query("Metadata_model_type == 'final'")
    .drop(columns=["p_value"])
    .pivot(index=metadata_columns, columns="phenotype", values="comparison_metric_value")
    .reset_index()
)

jump_wide_final_df.to_csv(final_jump_phenotype_file, sep="\t", index=False)

print(jump_wide_final_df.shape)
jump_wide_final_df.head()

(16179, 22)


phenotype,Metadata_Plate,treatment,treatment_type,Cell_type,Time,Metadata_Well,cell_count,ADCCM,Anaphase,Apoptosis,...,Grape,Hole,Interphase,Large,Metaphase,MetaphaseAlignment,OutOfFocus,Polylobed,Prometaphase,SmallIrregular
0,BR00116991,1-EBIO,compound,A549,24,P06,1473,0.046737,0.048754,0.050341,...,0.042167,0.028808,0.077078,0.067620,0.031090,0.038145,0.058110,0.024753,0.035821,0.034527
1,BR00116991,1-octanol,compound,A549,24,I24,1111,0.031986,0.085904,0.099348,...,0.067738,0.031380,0.088518,0.102193,0.064525,0.032058,0.099526,0.025510,0.031990,0.066391
2,BR00116991,"2,5-furandimethanol",compound,A549,24,G12,1153,0.027638,0.094782,0.084658,...,0.058770,0.039559,0.061289,0.100094,0.063818,0.040280,0.077559,0.046654,0.044400,0.058264
3,BR00116991,2-Oleoylglycerol,compound,A549,24,H02,1451,0.041859,0.084837,0.078716,...,0.063077,0.022829,0.049826,0.075192,0.067639,0.034138,0.074687,0.033886,0.027407,0.062492
4,BR00116991,4-CMTB,compound,A549,24,C01,1570,0.112659,0.078071,0.070801,...,0.105355,0.051031,0.045419,0.050727,0.111810,0.111739,0.092894,0.089923,0.039914,0.068263


In [14]:
jump_wide_shuffled_df = (
    jump_pred_df
    .query("Metadata_model_type == 'shuffled'")
    .drop(columns=["p_value"])
    .pivot(index=metadata_columns, columns="phenotype", values="comparison_metric_value")
    .reset_index()
)

jump_wide_shuffled_df.to_csv(shuffled_jump_phenotype_file, sep="\t", index=False)

print(jump_wide_shuffled_df.shape)
jump_wide_shuffled_df.head()

(16179, 22)


phenotype,Metadata_Plate,treatment,treatment_type,Cell_type,Time,Metadata_Well,cell_count,ADCCM,Anaphase,Apoptosis,...,Grape,Hole,Interphase,Large,Metaphase,MetaphaseAlignment,OutOfFocus,Polylobed,Prometaphase,SmallIrregular
0,BR00116991,1-EBIO,compound,A549,24,P06,1473,0.035732,0.047889,0.049714,...,0.027401,0.026632,0.057831,0.019390,0.032145,0.044006,0.050428,0.034754,0.031455,0.053313
1,BR00116991,1-octanol,compound,A549,24,I24,1111,0.040171,0.058095,0.064366,...,0.028038,0.031712,0.025602,0.039935,0.031761,0.032016,0.037037,0.021930,0.023784,0.046658
2,BR00116991,"2,5-furandimethanol",compound,A549,24,G12,1153,0.023798,0.055775,0.066432,...,0.024953,0.052325,0.031039,0.035398,0.023379,0.025458,0.044938,0.024937,0.057878,0.058678
3,BR00116991,2-Oleoylglycerol,compound,A549,24,H02,1451,0.024628,0.028920,0.061039,...,0.015462,0.039040,0.030507,0.045197,0.032572,0.033012,0.046498,0.051909,0.050479,0.057157
4,BR00116991,4-CMTB,compound,A549,24,C01,1570,0.059344,0.084141,0.062821,...,0.124347,0.046286,0.064387,0.016084,0.060213,0.075130,0.103736,0.039124,0.039918,0.021440


## Apply UMAP to phenotypic profiles

In [15]:
umap_random_seed = 123
umap_n_components = 2

feature_columns = jump_wide_final_df.drop(columns=metadata_columns).columns.tolist()
print(len(feature_columns))

15


In [16]:
umap_with_metadata_df = umap_phenotype(
    phenotype_df=jump_wide_final_df,
    feature_columns=feature_columns,
    metadata_columns=metadata_columns,
    n_components=umap_n_components,
    random_seed=umap_random_seed,
    model_type="final"
)

/home/gway/miniconda3/envs/phenotypic_profiling_v3/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [17]:
umap_shuffled_with_metadata_df = umap_phenotype(
    phenotype_df=jump_wide_shuffled_df,
    feature_columns=feature_columns,
    metadata_columns=metadata_columns,
    n_components=umap_n_components,
    random_seed=umap_random_seed,
    model_type="shuffled"
)

/home/gway/miniconda3/envs/phenotypic_profiling_v3/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [18]:
# Output file
umap_full_df = pd.concat([
    umap_with_metadata_df,
    umap_shuffled_with_metadata_df
], axis="rows")

umap_full_df.to_csv(jump_umap_file, sep="\t", index=False)

print(umap_full_df.shape)
umap_full_df.head()

(32358, 10)


,Metadata_Plate,treatment,treatment_type,Cell_type,Time,Metadata_Well,cell_count,UMAP0,UMAP1,model_type
0,BR00116991,1-EBIO,compound,A549,24,P06,1473,12.305723,6.603716,final
1,BR00116991,1-octanol,compound,A549,24,I24,1111,9.710662,4.139770,final
2,BR00116991,"2,5-furandimethanol",compound,A549,24,G12,1153,10.523870,4.440982,final
3,BR00116991,2-Oleoylglycerol,compound,A549,24,H02,1451,11.066898,4.891216,final
4,BR00116991,4-CMTB,compound,A549,24,C01,1570,7.409092,5.019341,final
